### Interval Data (Proportion and Probability) by XAI Conditions for SPSS analysis

In [1]:
import pandas as pd
import numpy as np

def get_XAI(idd):
    if 'X' in idd:
        return 1
    else:
        return 0

def get_spss_data(tracker_results):
    tracker_results['XAI_value'] = tracker_results['ID'].apply(get_XAI)
    # calculate 'elmo_probability' and handle division by zero or infinite results
    tracker_results['elmo_probability'] = np.where(
        tracker_results['fixation_counts'] == 0, 
        0, # if there are no fixations in computer screen, the probability is 0 (invalid)
        tracker_results['fixation_counts_elmo'] / tracker_results['fixation_counts']
    )

    # calculate 'elmo_proportion' and handle division by zero or infinite results
    tracker_results['elmo_proportion'] = np.where(
        tracker_results['fixation_durations'] == 0, 
        0, # if there are no fixations in computer screen, the proportion is 0 (invalid)
        tracker_results['fixation_durations_elmo'] / tracker_results['fixation_durations']
    )

interval_data = pd.read_csv('mushroom_data/interval_data_by_tasks_with_average_durations.csv')
get_spss_data(interval_data)
interval_data.to_csv('mushroom_data/final_interval_data_spss.csv', index=False)

interval_data_xai = pd.read_csv('mushroom_data/interval_data_XAI_removed.csv')
get_spss_data(interval_data_xai)
interval_data_xai.to_csv('mushroom_data/final_interval_data_XAI_removed_spss.csv', index=False)


### Interval Data By Trust Phases for SPSS analysis


In [2]:
# interval_data = pd.read_csv('mushroom_data/final_interval_data_spss.csv')

interval_data = pd.read_csv('mushroom_data/final_interval_data_XAI_removed_spss.csv')
# phase 1 = early trust build, phase 2 = early trust recovery, phase 3 = late trust recovery, phase 4 = ending effect
# Tasks 1-5 
# check if task is 1 to 5, then create a new column with 0 
interval_data['phase'] = np.where(interval_data['task'].isin([1,2,3,4,5]), 0, 1)
# Tasks 6-10
# check if task is 6 to 10, then create a new column with 1
interval_data['phase'] = np.where(interval_data['task'].isin([6,7,8,9,10]), 1, interval_data['phase'])
# Tasks 11-13
# check if task is 11 to 13, then create a new column with 2
interval_data['phase'] = np.where(interval_data['task'].isin([11,12,13]), 2, interval_data['phase'])
# Tasks 14-15
# check if task is 14 to 15, then create a new column with 3
interval_data['phase'] = np.where(interval_data['task'].isin([14,15]), 3, interval_data['phase'])

# tasks 0 and -1 (irrelevant tasks)
interval_data['phase'] = np.where(interval_data['task'].isin([0, -1]), 4, interval_data['phase'])
# save the data 
interval_data.to_csv('mushroom_data/interval_data_XAI_removed_spss.csv', index=False)

### Add all reliance data

In [19]:
# 0 underreliance and no XAI
# 11 no underreliance and no XAI
# 1 underreliance and XAI
# 10 no underreliance and XAI

# 2 overreliance and no XAI
# 12 no overreliance and no XAI
# 3 overreliance and XAI
# 13 no overreliance and XAI

# 4 reliance and no XAI
# 14 no reliance and no XAI
# 5 reliance and XAI
# 15 no reliance and XAI

underreliance = pd.read_csv('gaze_data/reliance_groups/under_reliance_global.csv')
underreliance["condition"] = underreliance["condition_underreliance"]

overreliance = pd.read_csv('gaze_data/reliance_groups/over_reliance_global.csv')

# convert to spss format
def get_overreliance(condition):
    if condition == 0:
        return 2
    elif condition == 1:
        return 3
    elif condition == 10:
        return 12
    elif condition == 11:
        return 13

def get_reliance(condition):
    if condition == 0:
        return 4
    elif condition == 1:
        return 5
    elif condition == 10:
        return 14
    elif condition == 11:
        return 15

overreliance["condition"] = overreliance["condition_overreliance"].apply(get_overreliance)

reliance = pd.read_csv('gaze_data/reliance_groups/reliance_global.csv')
reliance["condition"] = reliance["condition_reliance"].apply(get_reliance)

global_results = pd.concat([underreliance, overreliance, reliance])
global_results["proportion_elmo"] = global_results["fixation_durations_elmo"] / global_results["fixation_durations"]
global_results["probability_elmo"] = global_results["fixation_counts_elmo"] / global_results["fixation_counts"]
# save global results
global_results.to_csv('gaze_data/reliance_groups/reliance_global_results.csv', index=False)
